In [3]:
import sys
!{sys.executable} -m pip install scipy

import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
from ta import add_all_ta_features
from ta.utils import dropna


# Add ta features filling NaN values


#from scipy.stats import percentileofscore
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
finarr=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:.2%}"
wind=14
stklst=[]
for i in range(0,2):
    stk=sip[i]
    stklst.append(stk)
    #print(stklst)
    #df = yf.download(stk, start_date, end_date,interval=intr).dropna()
    #print(stk)
    dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
    dfd['Open1']=dfd.Open.shift(1)
    dfd['Close1']=dfd.Close.shift(1)
    dfd['High1']=dfd.High.shift(1)
    dfd['Low1']=dfd.Low.shift(1)
    dfd['Volume1']=dfd.Volume.shift(1)
    dfd['Change']=dfd.Close-dfd.Close1
    
    
    dfd = add_all_ta_features(
    dfd, open="Open1", high="High1", low="Low1", close="Close1", volume="Volume1", fillna=True)
    dfd=dfd.dropna()
    temp1=[]
    for j in range (12,len(dfd.columns)):
        ave=np.average(dfd.iloc[:,j])
        temp1.append(ave)
    #print(dfd.head)
    #print(temp1)
    
    dfdtop=dfd[dfd['Change']>=np.percentile(dfd.Change,99)]
    temp2=[]
    for j in range (12,len(dfd.columns)):
        ave=np.average(dfdtop.iloc[:,j])
        temp2.append(ave)
        
    temp3=[]
    for j in range (12,len(dfd.columns)):
        ave=np.average(dfdtop.iloc[:,j])
        perc=stats.percentileofscore(dfd.iloc[:,j],ave)
        temp3.append(perc)
        
    
    #fin=np.subtract(temp2,temp1)
    #fin=np.divide(fin,temp1)
    #print(fin)
    #fin=pd.Series(fin)
    fin=pd.Series(temp3)
    
    finarr=finarr.append(fin,ignore_index=True)
    

   # print(dfd.columns[11])
finarr=finarr.transpose()
finarr=finarr.set_index((dfd.columns[12:len(dfd.columns)]))
finarr.columns=[stklst]
#pd.options.display.float_format = '{:.2%}'.format
print((finarr))
#print(dfd.columns[12])

fin=np.array([])
for k in range (0,len(finarr.columns)):
    temp1=(np.nanmean(finarr.iloc[:,k]))    
    fin=np.append(fin,temp1)
dff=pd.DataFrame(fin)   

#dff.index=(dfd.columns[12:len(dfd.columns)])
print(dff)
dff.to_csv('ta_rev.csv')




[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/srv/conda/envs/notebook/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


[*********************100%***********************]  1 of 1 completed
                           MMM        ABT
volume_adi           76.997349  89.341159
volume_obv           78.000757  91.442635
volume_cmf           43.373722  37.826581
volume_fi            10.886028  33.869746
volume_mfi           50.605831  50.208254
...                        ...        ...
momentum_ppo_signal  73.778872  71.809920
momentum_ppo_hist    67.834154  63.593336
others_dr            29.988641  16.016660
others_dlr           28.814843  15.543355
others_cr            50.009466  50.009466

[83 rows x 2 columns]
           0
0  56.630612
1  61.791695
